# First Steps: 
Change the working directory to TRINA so you can do the imports without too much thinking

In [ ]:
import os
# os.chdir('~/TRINA')
home = os.path.expanduser("~")

trina_dir = os.path.join(home,'TRINA')
os.chdir(trina_dir)
print(trina_dir)

In [ ]:
import sys
sys.version_info

# We now import the relevant libraries for this tutorial:
Jarvis and the Sensor Module and motion_client and Klampt

In [ ]:
from Jarvis import Jarvis
from SensorModule import Camera_Robot
from Motion import MotionClient
from klampt import WorldModel,Geometry3D


# Testing your module in the jarvis Environment

The first thing you must remember is that in order to test this you will need to create two things: One Jarvis Instance and one Camera_Robot instance.

In order to create the Camera_Robot Instance you will need two elements: 
One motion_client instance, so that the module can query the robot's position (useful in simulation and in real life to perform the transforms to the cameras) and one Klampt world instance, which is useful in simulation (so that the robot can simulate sensor outputs)

In this tutorial, we will use one of our sample worlds, './Motion/data/TRINA_world_anthrax_PointClick.xml'
## starting the motion_client instance

In [ ]:
# in this example, we are running everything locally, so the robot's IP is localhost and our mode is "Kinematic"
robot_ip =  'http://localhost:8080'
mode = 'Physical'
# we also wish to be able to control all the components, so our components are:
components = ['right_limb']

# before running this part, make sure you already have a motion_server.py and command_server.py running!
robot = MotionClient(address = robot_ip)
robot.restartServer(mode = mode, components = components,codename = 'anthrax_lowpoly')

# loading the world and starting sensor_module instance

In [ ]:
world_file = './Motion/data/TRINA_world_anthrax_PointClick.xml'
# we load the world we wish to simulate
world = WorldModel()
world.readFile(world_file )

# then create a sensor_module instance using the motionClient instance and the world we created
sensor_module = Camera_Robot(robot = robot,world = world, cameras =['realsense_overhead','zed_overhead'],ros_active = False, use_jarvis = False, mode = mode)

## Obtaining images:

In [ ]:
result = sensor_module.get_rgbd_images()

In [ ]:
# displaying both images:
from matplotlib import pyplot as plt

for camera in result.keys():
    print('These are the images from camera :',camera)
    plt.imshow(result[camera][0])
    plt.show()
    plt.imshow(result[camera][1])
    plt.show()

## Obtaining Point Clouds

In [ ]:
import open3d as o3d

pcds = sensor_module.get_point_clouds()

In [ ]:
for camera in result.keys():
    print('This is the point cloud from camera :',camera)
    o3d.visualization.draw_geometries([pcds[camera]])

# Jarvis instance creation:

You now have all you need to create your own Jarvis instance and start using it exactly as you wold within your own module! The only thing of note is that, by default, the command server ignores commands of inactive modules, which, in theory, had to be activated by the UI. In order to remedy that, there is a set of module names that are always set as active. They are: 'UI', 'devel' and 'debug'. While testing your module, it is a good idea to set your module name in jarvis to be 'devel'.

In [ ]:
jarvis = Jarvis(str('devel'),sensor_module)

This jarvis instance should now behave exactly as it would if you had received if as an argument to your module - except it bypasses the activity status check because of its name.

In [ ]:
jarvis.setBaseVelocity([0,0])

# End of Tutorial